In [ ]:
import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

main_directory = "Runs"

directories = [
    "straightLine",
    "smallObstacle",
    "oppositeMultipleAgents",
    "oppositeCircleAgents",
    "oppositeAgents",
    "narrowCoridorsOppositeNoNavmeshScenario",
    "narrowCoridorOpposite",
    "cornerSingle",
]

runs = [
    name
    for name in os.listdir(main_directory)
    if os.path.isdir(os.path.join(main_directory, name))
]

runs_dfs = []

In [ ]:
for run in runs:
    new_directory = main_directory + "/" + run + "/" + "Averages"
    if not os.path.exists(new_directory):
        os.makedirs(new_directory)
    scenarios_dfs = []
    for directory in directories:
        path = main_directory + "/" + run + "/" + directory

        # Get list of CSV files in the directory
        csv_files = [
            os.path.join(path, file)
            for file in os.listdir(path)
            if file.endswith(".csv")
        ]
        # Read CSV files into a list of DataFrames
        dfs = [pd.read_csv(file) for file in csv_files]
        # Add agent_id column to each DataFrame based on filename
        for i, df in enumerate(dfs):
            df["agent_id"] = i
            
        dfs = pd.concat(dfs, ignore_index=True)
        dfs["run_id"] = run
        dfs["scenario"] = directory
        
        scenarios_dfs.append(dfs)
    scenarios_dfs = pd.concat(scenarios_dfs, ignore_index=True)
    runs_dfs.append(scenarios_dfs)


In [ ]:
# Prints violin plots for each scenario in each run.
for run_df in runs_dfs:
    for scenario in directories:
        scenario_df = run_df[run_df["scenario"] == scenario]
        scenario_df = scenario_df.drop(columns=["scenario", "agent_id", "run_id"])
        for column in scenario_df.columns:
            sns.violinplot(data=scenario_df[column])
            plt.title(run_df["run_id"].iloc[0] + " " + scenario)
            plt.show()

In [ ]:
# Prints violin plots for each scenario-column combination. X axis is the run_id
scenarios_data = []
for scenario in directories:
    extracted_data = [df[df["scenario"] == scenario] for df in runs_dfs]
    extracted_data = pd.concat(extracted_data, ignore_index=True)
    scenarios_data.append(extracted_data)

for scenario in scenarios_data:
    for column in scenario.columns:
        if column == "run_id" or column == "scenario" or column == "agent_id":
            continue
        sns.violinplot(x="run_id", y=scenario[column], data=scenario)
        plt.title(f"{scenario['scenario'][0]} - {column}")
        plt.xticks(range(len(runs)), [f'Run {runs[i]}' for i in range(len(runs))], rotation=45)
        plt.show()
        # plt.savefig(main_directory + "/Averages/" + scenario + "_" + column + ".png")
        plt.close()

print(extracted_data)